#Faster R-CNN with Detectron2

This tutorial was referenced from this [blog](https://towardsdatascience.com/how-to-train-detectron2-on-custom-object-detection-data-be9d1c233e4) by Jacob Solawetz. 

It implements the new [Detectron2 Library](https://ai.facebook.com/blog/-detectron2-a-pytorch-based-modular-object-detection-library-/) by facebook. This notebook shows training on **your own custom objects** for object detection.

It is worth noting that the Detectron2 library goes far beyond object detection, supporting semantic segmentation, keypoint detection, mask, and densepose.


### Accompanying Blog Post

We recommend that you follow along in this notebook while reading the blog post on [how to train Detectron2](https://blog.roboflow.ai/how-to-train-detectron2/), concurrently.

### Steps Covered in this Tutorial

In this tutorial, we will walk through the steps required to train Detectron2 on your custom objects. We use a [public blood cell detection dataset](https://public.roboflow.ai/object-detection/bccd), which is open source and free to use. You can also use this notebook on your own data.

To train our detector we take the following steps:

* Install Detectron2 dependencies
* Prepare custom Detectron2 object detection data
* Visualize Detectron2 training data
* Write our Detectron2 Training configuration
* Run Detectron2 training
* Evaluate Detectron2 performance
* Run Detectron2 inference on test images



### **About Roboflow**

[Roboflow](https://roboflow.ai) enables teams to deploy custom computer vision models quickly and accurately. Convert data from to annotation format, assess dataset health, preprocess, augment, and more. It's free for your first 1000 source images.

#### ![Roboflow Workmark](https://i.imgur.com/WHFqYSJ.png)

# Install Detectron2 Dependencies
The code below for installation is the most updated as of the time of writing. If any errors occur, please refer to the official Detectron2 Google Colab installation guide [here](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5).

In [ ]:
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
Requirement already up-to-date: torch==1.5 in /usr/local/lib/python3.6/dist-packages (1.5.0+cu101)
Requirement already up-to-date: torchvision==0.6 in /usr/local/lib/python3.6/dist-packages (0.6.0+cu101)
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-p5wbutmg
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-p5wbutmg
  Created wheel for pycocotools: filename=pycocotools-2.0-cp36-cp36m-linux_x86_64.whl size=266987 sha256=5ae4e8f14bd2e76bf2b052118ae48d245f35d6c72cd2fb3cb887fdc0c2f9f876
  Stored in directory: /tmp/pip-ephem-wheel-cache-rp4yh5sd/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


1.5.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

#Import Relevant Libraries

In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

# Prepare  a custom  dataset with Roboflow


## Annotating the Dataset
You can annotate your dataset with the [VGG Annotator online](http://www.robots.ox.ac.uk/~vgg/software/via/via_demo.html). It is intuitive and easy to use. However, do take note of the following precautions when annotating your images:
1. Draw only **polygons**, as rectangles are will not produce X-Y coordinates needed during training.

2. If your objects are close to the edge of the image, please take precaution to draw your boundaries within the image, otherwise an error will be produced during training.

3. Save both coco and json formats of your annotations just in case. When you want to load your annotations into VGG again, using the json format file will allow it to be more accurate, as coco formats do not preserve the names of the objects within your images.

To my knowledge, Roboflow accepts both JSON and COCO type formats. 
If you choose to use *COCO format* annotations, please take note of the following: 
Whenannotating your images, you would have realised that the categories that can be chosen are limited. In order to edit classes and categories of your images, scroll to the bottom of your coco file, and edit according to the example below:

In [ ]:
#example of how to edit last paragrah of coco file
"categories": [
    {"supercategory": "person","id": 1,"name": "person"},
    {"supercategory": "vehicle","id": 2,"name": "bicycle"},
    {"supercategory": "vehicle","id": 3,"name": "car"},
    {"supercategory": "vehicle","id": 4,"name": "motorcycle"},
    {"supercategory": "vehicle","id": 5,"name": "airplane"},
    ...
    {"supercategory": "indoor","id": 89,"name": "hair drier"},
    {"supercategory": "indoor","id": 90,"name": "toothbrush"}
]

## Uploading and Importing Data from Roboflow
With your dataset and annotations ready, you may head over to [Roboflow](robolow.ai) to upload your dataset and annotations. 

Take note that Roboflow will randomly split your dataset to training, validation and test set (you will be able to choose the ratio). 

The interface of Roboflow is very easy to use, so I shall not explain much of it. You may also add in augmentation through the Roboflow interface if you want. 

To receive the url to download your dataset to Google Colab, simply click the "Generate" button at the top right hand corner of the page, and choose "Show download code". It should generate a piece of code similar to the one below to download your very own dataset.

In [ ]:
!curl -L "https://app.roboflow.ai/ds/KcHabaG0Jx?key=lYhi5aSk0J" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   887  100   887    0     0    788      0  0:00:01  0:00:01 --:--:--   788
100 20.2M  100 20.2M    0     0  7055k      0  0:00:02  0:00:02 --:--:-- 15.3M
Archive:  roboflow.zip
 extracting: test/es15_jpg.rf.6ea5b45cd9b99fa522ab06ea177f7f47.jpg  
 extracting: test/download_jpg.rf.7a8604f40c51c46ccc33fe2b48fa8c1e.jpg  
 extracting: test/l-10_jpg.rf.20437d34d6e9a91af8d5b12b6be831cb.jpg  
 extracting: test/DOR_S3_53_JPG.rf.2f6901b198392ea6a5cc110b77c42712.jpg  
 extracting: test/e57_jpg.rf.41c6c87a2f7a07d13c1ae34f6f86c0e2.jpg  
 extracting: test/autodoor26_jpg.rf.27650ae564dff31be247d3dad48e48b0.jpg  
 extracting: test/l-7_jpg.rf.4b51a4e2a3eac759f05966e723e27803.jpg  
 extracting: test/es19_jpg.rf.574f1b24fc0b1301c1a76f73bf53712a.jpg  
 extracting: test/e38_jpg.rf.1eb60e31cba62daaa68e1229683e28b6.jpg  
 extracting: test/l-9_jpg.rf

Register the doorway dataset to detectron2, following the [detectron2 custom dataset tutorial](https://detectron2.readthedocs.io/tutorials/datasets.html).

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("doorway_train", {}, "/dataset/train/annotations.json", "/dataset/train/images")
register_coco_instances("doorway_val", {}, "/dataset/val/annotations.json", "/dataset/val/images")
register_coco_instances("doorway_test", {}, "/dataset/predict/annotations.json", "/dataset/predict/images")


To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:


In [ ]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("doorway_train")
dataset_dicts = DatasetCatalog.get("doorway_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7febf6b3dda0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 962, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 942, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 124, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 50, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.6/multiprocessing/popen_fork.py", line 28, in poll
    pid, sts = os.waitpid(self.pid, flag)
KeyboardInterrupt: 

KeyboardInterrupt



#Configure & Train the model
Before we start training the model, there are a few configurations that must be done. Firstly, edit the *MODEL.ROI_HEADS.NUM_CLASSES* variable to the same number of classes that the dataset has plus 1 to incude the background as a class. 

The variables that can be changed can be found in the *configs folder* in the Detectron2 folder that you have cloned earlier.  The config files houses all the default configurations for your model, and making changes in the *directly in the cell below will overwrite the default configuration.

The configurations of your model depends greatly on the kind of data you would like to train your model on. Hence, I am unable to give you a definite answer on what is the "best" configuration for your model. 

 It takes ~30 minutes to 1.5 hours to train 2000+ iterations on Colab's GPU.

In [ ]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
#from .detectron2.tools.train_net import Trainer
#from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
#from detectron2.evaluation.coco_evaluation import COCOEvaluator
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001


cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05




cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5 #your number of classes + 1

cfg.TEST.EVAL_PERIOD = 500


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/14 07:33:42 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Unable to load 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (6, 1024) in the model!
Unable to load 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (6,) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (20, 1024) in the model!
Unable to load 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (20,) in the model!


[07/14 07:33:43 d2.engine.train_loop]: Starting training from iteration 0
[07/14 07:35:24 d2.utils.events]:  eta: 2:09:49  iter: 19  total_loss: 2.498  loss_cls: 1.619  loss_box_reg: 0.816  loss_rpn_cls: 0.054  loss_rpn_loc: 0.016  time: 5.0573  data_time: 0.0334  lr: 0.000020  max_mem: 10444M
[07/14 07:37:02 d2.utils.events]:  eta: 2:07:33  iter: 39  total_loss: 2.069  loss_cls: 1.258  loss_box_reg: 0.800  loss_rpn_cls: 0.027  loss_rpn_loc: 0.013  time: 4.9909  data_time: 0.0132  lr: 0.000040  max_mem: 10444M
[07/14 07:38:42 d2.utils.events]:  eta: 2:05:37  iter: 59  total_loss: 1.748  loss_cls: 0.865  loss_box_reg: 0.787  loss_rpn_cls: 0.036  loss_rpn_loc: 0.013  time: 4.9860  data_time: 0.0153  lr: 0.000060  max_mem: 10444M
[07/14 07:40:22 d2.utils.events]:  eta: 2:03:31  iter: 79  total_loss: 1.599  loss_cls: 0.690  loss_box_reg: 0.883  loss_rpn_cls: 0.012  loss_rpn_loc: 0.009  time: 4.9878  data_time: 0.0179  lr: 0.000080  max_mem: 10444M
[07/14 07:42:02 d2.utils.events]:  eta: 2:

#Save the Model


In [ ]:
from detectron2.modeling import build_model
import torch
model = build_model(cfg)
torch.save(model.state_dict(), '/content/output/model.pth')

#Load the Model
To use the model for inference, 

In [ ]:
model.load_state_dict(torch.load( '/content/output/model.pth', map_location='cpu'))

#Evaluating the Model
Now, we perform inference with the trained model on the doorway dataset. First, let's create a predictor using the model we just trained.

##Average Precision (AP) Values on Test Dataset

These values are important in showing how well the model is doing overall, and for each individual classes. This will be crucial in spotting any overfitting tendencies within the model, and how well it fairs with different IoU levels. You can read more about AP values and intersections over unions (IoU) [here](https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173). Usually, a score of around 80% for AP50 would be good enough for me. To improve on your accuracy, consider tweaking the configurations, or adding new data to your dataset.

In [ ]:
#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

WARNING [07/14 09:50:03 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[07/14 09:50:03 d2.data.datasets.coco]: Loaded 47 images in COCO format from /content/test/_annotations.coco.json
[07/14 09:50:03 d2.data.common]: Serializing 47 elements to byte tensors and concatenating them all ...
[07/14 09:50:03 d2.data.common]: Serialized dataset takes 0.01 MiB
[07/14 09:50:03 d2.evaluation.evaluator]: Start inference on 47 images
[07/14 09:50:09 d2.evaluation.evaluator]: Inference done 11/47. 0.5448 s / img. ETA=0:00:19
[07/14 09:50:15 d2.evaluation.evaluator]: Inference done 21/47. 0.5483 s / img. ETA=0:00:14
[07/14 09:50:20 d2.evaluation.evaluator]: Inference done 31/47. 0.5490 s / img. ETA=0:00:08
[07/14 09:50:26 d2.evaluation.evaluator]: Inference done 41/47. 0.5491 s / img. ETA=0:00:03
[07/14 09:50:29 d2.evaluation.evaluator]: Total inference time: 0:00:23.246380 (0.553485 s / img per device, on 1 devices)
[07/14 09:50:29

OrderedDict([('bbox',
              {'AP': 60.20255444723557,
               'AP-Training': nan,
               'AP-door': 54.89893469397873,
               'AP-escalator': 55.42713886773293,
               'AP-lift': 74.74090266169473,
               'AP-stairs': 55.74324156553586,
               'AP50': 85.18898339209584,
               'AP75': 70.70750850388683,
               'APl': 60.39732534584296,
               'APm': 64.99999999999999,
               'APs': nan})])

##Visual Evaluation on Test Dataset
Here, we randomly select several samples to visualize the prediction results.

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])


Output hidden; open in https://colab.research.google.com to view.